
# Dose-Response Analysis of Sf9 Cells Exposed to Bacillus thuringiensis Insecticidal Protein


This script is designed for image data analysis of cell segmentation in bright field microscopy images of Sf9 cells exposed to *Bacillus thuringiensis* insecticidal protein.

## Overview:
- Script created by Adan Guerrero (adan.guerrero@ibt.unam.mx).
- This script analyzes the dose-response relationship between cell conditions and exposure to the protein.
- It requires the `summary_report.csv` file generated by the script `240516-cyto3.ipynb`.

## Input:
- The `summary_report.csv` file contains cell segmentation data, including the number of ROIs, median area, and concentration values.

## Output:
- Tables with metrics for each sample (e.g., number of ROIs, median area, concentration).
- Dose-response plots visualizing the relationship between protein concentration and cell response.

## Dependencies:
- Python 3.x
- Libraries: `statsmodels`, `seaborn`, `pandas`
- Install the required libraries via:
  `pip install statsmodels seaborn pandas`

## Execution:
1. Ensure that the `summary_report.csv` file is available in the appropriate directory.
2. Run the script to process the data and generate dose-response plots.


In [ ]:
#!pip install statsmodels seaborn

In [ ]:
################ Libraries
import os
import pandas as pd
import numpy as np
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.backends.backend_pdf
import re
from scipy.stats import ttest_ind

################ Fuctions
# Extract the concentration from the filenames
def extract_concentration(filename):
    if "control" in filename:
        return 0.0
    match = re.search(extraction_string, filename)
   # match = re.search(r'\b(\d+(\.\d+)?)\b', filename)
    if match:
        return float(match.group(1))
    return None

###          Cry1Fa

folder_path = "Cry1Fa  toxin and protoxin/Cry1Fa Toxin/72h/"
conc = [0.5, 1, 1.5, 2, 2.5, 3, 3.75]
extraction_string = r'72h tox Cry1Fa (\d+(\.\d+)?)'
es = '72h tox Cry1Fa'
###


#folder_path = "Cry1Fa  toxin and protoxin/Cry1Fa Protoxin/72h"
#conc = [1, 1.5, 2, 2.5, 3, 3.75]
#extraction_string = r'72h Cry1Fa protoxin (\d+(\.\d+)?)'
#es= '72h Cry1Fa protoxin'

################

########### Vip 3


#folder_path = "Vip3Aa toxin and protoxin/TOXIN Vip3/72 h toxin"
#conc = [0.6, 1.25, 2.5, 5, 7.5, 10]
#extraction_string = r' (\d+(\.\d+)?)'
#es= '72h Vip3Aa toxin'

#folder_path = "Vip3Aa toxin and protoxin/PROTOX Vip3/72h protoxin"
#conc = [5, 10, 20, 30, 40, 50]
#extraction_string = r' (\d+(\.\d+)?)'
#es= '72h Vip3Aa Protox'


############## Load data
file_path = 'summary_report.csv'
# Define the summary CSV file path
file_path = os.path.join(folder_path, file_path)
df = pd.read_csv(file_path)
df

,Filename,Number of ROIs,Median Area
0,72h control-1.jpg,260,653.0
1,72h tox Cry1Fa 1-2.jpg,100,982.5
2,72h tox Cry1Fa 0.5-2.jpg,152,697.5
3,72h tox Cry1Fa 1.5-3.jpg,92,1045.0
4,72h tox Cry1Fa 0.5-3.jpg,98,972.0
5,72h control-3.jpg,235,650.0
6,72h tox Cry1Fa 1-1.jpg,119,999.0
7,72h tox Cry1Fa 1.5-2.jpg,93,1173.0
8,72h tox Cry1Fa 1-3.jpg,101,991.0
9,72h tox Cry1Fa 0.5-1.jpg,125,951.0


In [ ]:
df['Filename'].str.extract(extraction_string)[0].astype(float)

0      NaN
1     1.00
2     0.50
3     1.50
4     0.50
5      NaN
6     1.00
7     1.50
8     1.00
9     0.50
10    1.50
11     NaN
12    2.50
13    2.00
14    2.50
15    2.00
16    2.50
17    3.00
18    2.00
19    3.75
20    3.00
21    3.75
22    3.75
23    3.00
Name: 0, dtype: float64

In [ ]:
# Extract the concentration values from the filename using the manually defined pattern
df['Concentration'] = df['Filename'].str.extract(extraction_string)[0].astype(float)

# Identify control conditions: filenames that do not contain 'tox 1Fa'
control_conditions = df[df['Filename'].str.contains(es) == False].copy()

# Add a Concentration column to control conditions and set it to NaN
control_conditions.loc[:, 'Concentration'] = float('nan')

# Add a Concentration column to control conditions and set it to a unique value for plotting
control_conditions['Concentration'] = 0  # Assuming 0 represents control condition


# Sort by concentration and filename for better readability
df = df.sort_values(by=['Concentration', 'Filename'])

df["Concentration"] = df["Filename"].apply(extract_concentration)
df

,Filename,Number of ROIs,Median Area,Concentration
9,72h tox Cry1Fa 0.5-1.jpg,125,951.0,0.50
2,72h tox Cry1Fa 0.5-2.jpg,152,697.5,0.50
4,72h tox Cry1Fa 0.5-3.jpg,98,972.0,0.50
6,72h tox Cry1Fa 1-1.jpg,119,999.0,1.00
1,72h tox Cry1Fa 1-2.jpg,100,982.5,1.00
8,72h tox Cry1Fa 1-3.jpg,101,991.0,1.00
10,72h tox Cry1Fa 1.5-1.jpg,89,1065.0,1.50
7,72h tox Cry1Fa 1.5-2.jpg,93,1173.0,1.50
3,72h tox Cry1Fa 1.5-3.jpg,92,1045.0,1.50
13,72h tox Cry1Fa 2-1.jpg,90,1099.5,2.00


In [ ]:
df[df['Filename'].str.contains(es) == False].copy()

,Filename,Number of ROIs,Median Area,Concentration
0,72h control-1.jpg,260,653.0,0.0
11,72h control-2.jpg,279,646.0,0.0
5,72h control-3.jpg,235,650.0,0.0


In [ ]:
 df.groupby("Concentration")["Number of ROIs"].apply(list).reset_index()

,Concentration,Number of ROIs
0,0.00,"[260, 279, 235]"
1,0.50,"[125, 152, 98]"
2,1.00,"[119, 100, 101]"
3,1.50,"[89, 93, 92]"
4,2.00,"[90, 99, 65]"
5,2.50,"[88, 80, 72]"
6,3.00,"[73, 70, 56]"
7,3.75,"[49, 53, 61]"


In [ ]:
# Group by concentration and list the number of ROIs for each concentration
df_grouped = df.groupby("Concentration")["Number of ROIs"].apply(list).reset_index()

# Ensure each concentration has three replicates
df_grouped["Replicate 1"] = df_grouped["Number of ROIs"].apply(lambda x: x[0] if len(x) > 0 else None)
df_grouped["Replicate 2"] = df_grouped["Number of ROIs"].apply(lambda x: x[1] if len(x) > 1 else None)
df_grouped["Replicate 3"] = df_grouped["Number of ROIs"].apply(lambda x: x[2] if len(x) > 2 else None)
# Drop the original "Number of ROIs" column
df_grouped = df_grouped.drop(columns=["Number of ROIs"])

# Calculate the average number of ROIs in the control group
average_control_rois = np.nanmean(df_grouped[df_grouped["Concentration"] == 0.0][['Replicate 1', 'Replicate 2', 'Replicate 3']].values.flatten())

# Calculate the mortality rate for each concentration
mortality_rates_corrected = []
for i, row in df_grouped.iterrows():
    if row["Concentration"] != 0.0:
        treated_rois = [row["Replicate 1"], row["Replicate 2"], row["Replicate 3"]]
        average_treated_rois = np.nanmean(treated_rois)
        mortality_rate = (average_control_rois - average_treated_rois) / average_control_rois * 100
        mortality_rates_corrected.append({"Dose": row["Concentration"], "Mortality": mortality_rate})
        print(average_treated_rois)

# Convert mortality rates to a DataFrame
mortality_df_corrected = pd.DataFrame(mortality_rates_corrected)

mortality_df_corrected

125.0
106.66666666666667
91.33333333333333
84.66666666666667
80.0
66.33333333333333
54.333333333333336


,Dose,Mortality
0,0.50,51.550388
1,1.00,58.656331
2,1.50,64.599483
3,2.00,67.183463
4,2.50,68.992248
5,3.00,74.289406
6,3.75,78.940568


In [ ]:
#np.mean([127, 142, 115, 193, 170, 173])

In [ ]:

# Group by concentration and list the number of ROIs for each concentration
df_grouped = df.groupby("Concentration")["Number of ROIs"].apply(list).reset_index()

# Ensure each concentration has three replicates
df_grouped["Replicate 1"] = df_grouped["Number of ROIs"].apply(lambda x: x[0] if len(x) > 0 else None)
df_grouped["Replicate 2"] = df_grouped["Number of ROIs"].apply(lambda x: x[1] if len(x) > 1 else None)
df_grouped["Replicate 3"] = df_grouped["Number of ROIs"].apply(lambda x: x[2] if len(x) > 2 else None)
# Drop the original "Number of ROIs" column
df_grouped = df_grouped.drop(columns=["Number of ROIs"])

# Calculate the average number of ROIs in the control group
average_control_rois = np.mean(df_grouped[df_grouped["Concentration"] == 0.0][['Replicate 1', 'Replicate 2', 'Replicate 3']].values.flatten())

# Calculate the mortality rate for each concentration
mortality_rates_corrected = []
for i, row in df_grouped.iterrows():
    if row["Concentration"] != 0.0:
        treated_rois = [row["Replicate 1"], row["Replicate 2"], row["Replicate 3"]]
        average_treated_rois = np.mean(treated_rois)
        mortality_rate = (average_control_rois - average_treated_rois) / average_control_rois * 100
        #if mortality_rate >= 0:  # Omit negative values
            #mortality_rates_corrected.append({"Dose": row["Concentration"], "Mortality": mortality_rate})
        mortality_rates_corrected.append({"Dose": row["Concentration"], "Mortality": mortality_rate})

# Convert mortality rates to a DataFrame
mortality_df_corrected = pd.DataFrame(mortality_rates_corrected)

# Probit transformation of the mortality rates (expressed as proportions)
mortality_df_corrected["Probit"] = norm.ppf(mortality_df_corrected["Mortality"] / 100)

# Fit a probit model using the dose values and probit-transformed mortality rates
dose_values_corrected = mortality_df_corrected["Dose"].values
probit_values_corrected = mortality_df_corrected["Probit"].values
probit_model_corrected = sm.OLS(probit_values_corrected, sm.add_constant(dose_values_corrected)).fit()

# Extract parameters from the model
intercept_corrected, slope_corrected = probit_model_corrected.params

# Calculate the dose corresponding to 50% mortality (Probit value = 0)
ic50_dose_corrected = -intercept_corrected / slope_corrected



#######################
# Filter the data for both experimental and control cases
df_filtered = pd.concat([df[df['Concentration'].isin(conc)], control_conditions])

# Sort by concentration and filename for better readability
df_filtered = df_filtered.sort_values(by=['Concentration', 'Filename'])

# Compute the mean control median area
control_mean_area = control_conditions['Median Area'].mean()

# Normalize the median area to the control mean area
df_filtered['Normalized Area'] = df_filtered['Median Area'] / control_mean_area

# Perform statistical tests (t-test) to compare each concentration against the control
p_values = []
for concentration in df_filtered['Concentration'].unique():
    if concentration == 0:
        continue  # Skip control
    experimental_data = df_filtered[df_filtered['Concentration'] == concentration]['Median Area']
    t_stat, p_val = ttest_ind(control_conditions['Median Area'], experimental_data)
    p_values.append((concentration, p_val))

# Determine significance levels
significance = [(0.0, 'NaN')]
for concentration, p_val in p_values:
    if p_val < 0.001:
        significance.append((concentration, '***'))
    elif p_val < 0.01:
        significance.append((concentration, '**'))
    elif p_val < 0.05:
        significance.append((concentration, '*'))
    else:
        significance.append((concentration, 'ns'))


# Convert the significance list to a DataFrame
significance_df = pd.DataFrame(significance, columns=['Concentration', 'Significance'])

# Compute means and standard errors
grouped = df_filtered.groupby('Concentration')['Normalized Area'].agg(['mean', 'sem']).reset_index()


grouped = pd.merge(grouped, significance_df, on='Concentration', how='right')

########################
#df
df_filtered

,Filename,Number of ROIs,Median Area,Concentration,Normalized Area
0,72h control-1.jpg,260,653.0,0.00,1.005131
11,72h control-2.jpg,279,646.0,0.00,0.994356
5,72h control-3.jpg,235,650.0,0.00,1.000513
9,72h tox Cry1Fa 0.5-1.jpg,125,951.0,0.50,1.463828
2,72h tox Cry1Fa 0.5-2.jpg,152,697.5,0.50,1.073628
4,72h tox Cry1Fa 0.5-3.jpg,98,972.0,0.50,1.496152
6,72h tox Cry1Fa 1-1.jpg,119,999.0,1.00,1.537712
1,72h tox Cry1Fa 1-2.jpg,100,982.5,1.00,1.512314
8,72h tox Cry1Fa 1-3.jpg,101,991.0,1.00,1.525398
10,72h tox Cry1Fa 1.5-1.jpg,89,1065.0,1.50,1.639302


In [ ]:
mortality_df_corrected

,Dose,Mortality,Probit
0,0.50,51.550388,0.038872
1,1.00,58.656331,0.218713
2,1.50,64.599483,0.374530
3,2.00,67.183463,0.444985
4,2.50,68.992248,0.495631
5,3.00,74.289406,0.652293
6,3.75,78.940568,0.804361


In [ ]:
grouped

,Concentration,mean,sem,Significance
0,0.00,1.000000,0.003121,NaN
1,0.50,1.344536,0.135775,ns
2,1.00,1.525141,0.007333,***
3,1.50,1.684454,0.061193,***
4,2.00,1.620575,0.076104,**
5,2.50,1.450744,0.025514,***
6,3.00,1.021293,0.101995,ns
7,3.75,0.737301,0.028004,***


In [ ]:
mortality_df_corrected.to_csv(os.path.join(folder_path, es+" mortality.csv"), index=False)
grouped.to_csv(os.path.join(folder_path, es+' cell size.csv'), index=False)

# Convert the value to a DataFrame
ic50_df = pd.DataFrame({'IC50 Dose': [ic50_dose_corrected]})

# Calculate the dose corresponding to 50% mortality (Probit value = 0)
ic50_df.to_csv(os.path.join(folder_path, es+" IC50.csv"), index=False)


print('DataFrames have been saved to mortality_df_corrected.csv and grouped.csv')

DataFrames have been saved to mortality_df_corrected.csv and grouped.csv


In [ ]:
ic50_dose_corrected

0.08952574569041022